In [1]:
import pandas as pd
import pyarrow
import openpyxl

In [2]:
df=pd.read_csv(r"C:\Repos\ufo_project\data\complete_cleaned.csv",
                sep=";",
                header=0
                  )
df = pd.read_parquet(r"C:\Repos\ufo_project\data\complete_cleaned.parquet")
df.head(3)

,observation_datestamp,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,us,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),None,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667


In [3]:
print(df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
Index: 88679 entries, 0 to 88678
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   observation_datestamp  88679 non-null  datetime64[ns]
 1   city                   88679 non-null  object        
 2   state_id               81270 non-null  object        
 3   country_id             87560 non-null  object        
 4   shape_type             88679 non-null  object        
 5   duration_in_sec        88679 non-null  float64       
 6   duration_string        85660 non-null  object        
 7   comments               88679 non-null  object        
 8   date_posted            88679 non-null  datetime64[ns]
 9   latitude               88679 non-null  float64       
 10  longitude              88679 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 8.1+ MB
None


In [4]:
df["observation_datestamp"] = pd.to_datetime(df["observation_datestamp"])
df['date']=df['observation_datestamp'].dt.date
df['date']=pd.to_datetime(df['date'],format="%Y-%m-%d")
df['year'] = df['date'].dt.year

In [5]:
df.drop(columns="observation_datestamp",inplace=True)

In [6]:
print(df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
Index: 88679 entries, 0 to 88678
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   city             88679 non-null  object        
 1   state_id         81270 non-null  object        
 2   country_id       87560 non-null  object        
 3   shape_type       88679 non-null  object        
 4   duration_in_sec  88679 non-null  float64       
 5   duration_string  85660 non-null  object        
 6   comments         88679 non-null  object        
 7   date_posted      88679 non-null  datetime64[ns]
 8   latitude         88679 non-null  float64       
 9   longitude        88679 non-null  float64       
 10  date             88679 non-null  datetime64[ns]
 11  year             88679 non-null  int32         
dtypes: datetime64[ns](2), float64(3), int32(1), object(6)
memory usage: 8.5+ MB
None


In [7]:
df.head(3)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949
1,lackland afb,tx,us,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949
2,chester (uk/england),None,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,1955-10-10,1955


In [8]:
df['state_id']=df['state_id'].str.upper()
df['country_id']=df['country_id'].str.upper()
df['city']=df['city'].str.title()
df.head(3)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year
0,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949
1,Lackland Afb,TX,US,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949
2,Chester (Uk/England),None,GB,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,1955-10-10,1955


In [9]:
#check ile jest nawiasów w kolumnie city
nawiasy=df['city'].str.contains(r'\(.*?\)', case=False).sum()
print(nawiasy)

14540


In [10]:
#usuwanie nawiasow wraz z zawartoscia
df['city'] = df['city'].str.replace(r'\(.*?\)', '', regex=True)
df.head(3)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year
0,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949
1,Lackland Afb,TX,US,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949
2,Chester,None,GB,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,1955-10-10,1955


In [11]:
#check usuniecia
nawiasy=df['city'].str.contains(r'\(.*?\)', case=False).sum()
print(nawiasy)

0


In [12]:
#czy są same nawiasy otwierające
nawiasy_otwierajace=df['city'].str.contains(r'\(', case=False).sum()
print(nawiasy_otwierajace)

43


In [13]:
#usuwanie nawiasów otwierających wraz z zawartoscia
df['city'] = df['city'].str.replace(r'\(.*?', '', regex=True)

In [14]:
#check usuniecie nawiasow otwierajacych
nawiasy_otwierajace1 = df[df['city'].str.contains(r'\(', regex=True)]
print(nawiasy_otwierajace1)

Empty DataFrame
Columns: [city, state_id, country_id, shape_type, duration_in_sec, duration_string, comments, date_posted, latitude, longitude, date, year]
Index: []


In [15]:
#czy są same nawiasy zamykajace
nawiasy_zamykajace=df['city'].str.contains(r'\)', case=False).sum()
print(nawiasy_zamykajace)

35


In [16]:
nawiasy_zamykajace=df[df['city'].str.contains(r'\)', case=False)]
print(nawiasy_zamykajace)

                                                    city state_id country_id  \
5789                                     Unadilla  Near)       NE         US   
8194                            Ft. Resolution   Canada)       NT         US   
11993                                    Yorkeys Knob  )     None         AU   
14210                                   Fredericksburg )       TX         US   
17920                                                  )     None       None   
18597                                                  )       PA         US   
19537                                   Pacific Ocean )      None         PR   
20214                                        Woodville )       NC         US   
23437                                Sault Ste. Marie )        ON         CA   
27129                                           Espoo  )     None         FI   
33914                                           Piney  )       TN         US   
35771                                   

In [17]:
#ręczna poprawa powyższych danych
df.loc[8194,'country_id']='CA'
df.loc[5789,'city']='Unadilla'
df.loc[8194,'city']='Ft. Resolution'
df.loc[40609,'city']='Tel Aviv'
df.loc[40609,'country_id']='IL'
df.loc[52860,'city']='London'
df.loc[52860,'country_id']='GB'  
df.loc[45133,'city']='Iowa City'
df.loc[54713, 'city']='Phuket'
df.loc[54713, 'country_id']='TH'
df.loc[60128, 'city']=''
df.loc[60128, 'country_id']='AFG'
df.loc[75757,'city']=''   
df.loc[78575,'city']=''
df.loc[80683,'city']='Midwest'
df.loc[80683,'country_id']='US'

In [18]:
#usuwanie nawiasow zamykających + spacji przed
df['city'] = df['city'].str.replace(r' \)', '', regex=True)

In [19]:
#usuwanie nawiasow zamykających bez spacji
df['city'] = df['city'].str.replace(r'\)', '', regex=True)

In [20]:
#czy sa nawiasy zamykające
nawiasy_zamykajace = df[df['city'].str.contains(r'\)', regex=True)]
print(nawiasy_zamykajace)

Empty DataFrame
Columns: [city, state_id, country_id, shape_type, duration_in_sec, duration_string, comments, date_posted, latitude, longitude, date, year]
Index: []


In [21]:
df.head(30)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year
0,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949
1,Lackland Afb,TX,US,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949
2,Chester,None,GB,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,1955-10-10,1955
3,Edna,TX,US,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833,1956-10-10,1956
4,Kaneohe,HI,US,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611,1960-10-10,1960
5,Bristol,TN,US,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889,1961-10-10,1961
6,Penarth,None,GB,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2006-02-14,51.434722,-3.180000,1965-10-10,1965
7,Norwalk,CT,US,circle,1200.0,20 minutes,A bright orange color changing to reddish colo...,1999-10-02,41.117500,-73.408333,1965-10-10,1965
8,Pell City,AL,US,circle,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,2009-03-19,33.586111,-86.286111,1966-10-10,1966
9,Live Oak,FL,US,circle,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,2005-05-11,30.294722,-82.984167,1966-10-10,1966


In [22]:
df['city'] = df['city'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [23]:
import pycountry

In [24]:
#check gdzie brakuje country id
empty_country=df['country_id'].isnull().sum()
print(empty_country)

1114


In [25]:
empty=df[df['country_id'].isnull()]
print(empty)

                        city state_id country_id shape_type  duration_in_sec  \
18               Bermuda Nas     None       None      light             20.0   
102     Carnival Cruise Ship     None       None      light           1200.0   
126                  Bahamas     None       None       oval              0.0   
219                Greenland     None       None  rectangle              0.0   
553                 Viet Nam     None       None      light            600.0   
...                      ...      ...        ...        ...              ...   
88424      Germany Or France     None       None      cigar              0.0   
88459  Cozumel And Ocho Rios     None       None   triangle           2700.0   
88489               Thailand     None       None   changing            600.0   
88525                  Dubai     None       None     circle             15.0   
88623                Denmark     None       None      light            300.0   

      duration_string                  

In [26]:
df['city'] = df['city'].str.replace(r'\bNas\b', '', regex=True)

In [27]:
#funkcja do wyszukania nazwy kraju po country_id przy użyciu ISC code
def get_country_name(iso_code):
    if isinstance(iso_code, str):  # Sprawdzamy, czy kod jest łańcuchem znaków
        try:
            iso_code = iso_code.upper()
            country = pycountry.countries.get(alpha_2=iso_code)
            return country.name if country else None
        except KeyError:
            return None
    else:
        return None  # Dla NaN i innych nieprawidłowych danych zwracamy None

In [28]:
#nowa kolumna country_name, która korzysta z funkcji get_country_name
df['country_name'] = df['country_id'].apply(get_country_name)

In [29]:
df.head(2)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year,country_name
0,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949,United States
1,Lackland Afb,TX,US,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949,United States


In [30]:
#gdy country_name jest puste, dodajemy info z city
df['country_name'] = df['country_name'].fillna(df['city'])
df.head(2)

,city,state_id,country_id,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude,date,year,country_name
0,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949-10-10,1949,United States
1,Lackland Afb,TX,US,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,1949-10-10,1949,United States


In [31]:
new_order = ['date','year','city','state_id','country_id','country_name','shape_type','duration_in_sec','duration_string','comments','date_posted','latitude','longitude']

In [32]:
df = df[new_order]

In [33]:
#usunięcie niewidzialnych znaków z country_name
df['country_name'] = df['country_name'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [34]:
#check ile %obserwacji to pozycje z country_id NaN
liczba_nan = df['country_id'].isna().sum()

liczba_wszystkich = len(df)

procent_nan = (liczba_nan / liczba_wszystkich) * 100

print(f'Liczba wierszy z NaN w kolumnie country_id: {liczba_nan}')
print(f'To stanowi {procent_nan:.2f}% wszystkich obserwacji.')

Liczba wierszy z NaN w kolumnie country_id: 1114
To stanowi 1.26% wszystkich obserwacji.


In [43]:
#check ile %obserwacji to pozycje z city name ze znakami niewłaściwymi 
liczba_wys = df['city'].str.count(r'[^a-zA-Z\s]').sum()

liczba_wszystkich = len(df)

procent_nan = (liczba_wys / liczba_wszystkich) * 100

print(f'Liczba wierszy ze znakami obcymi to 6,3% w kolumnie city: {liczba_wys}')
print(f'To stanowi {procent_nan:.2f}% wszystkich obserwacji.')

Liczba wierszy ze znakami obcymi to 6,3% w kolumnie city: 7728
To stanowi 8.72% wszystkich obserwacji.


In [44]:
#usuwanie znaków znaków innych niż Literowe
df= df[~df['city'].str.contains(r'[^a-zA-Z\s]', na=False)]

In [45]:
#usuwanie znaków zapytania
df= df[~df['city'].str.contains(r'\?', na=False)]


In [46]:
def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_2  # Zwraca kod ISO alpha_2 (np. 'PL')
    except LookupError:
        return None  # Jeśli nie znajdzie kraju, zwraca None

# Uzupełnianie brakujących wartości w kolumnie country_id
df['country_id'] = df.apply(
    lambda row: get_country_code(row['country_name']) if pd.isna(row['country_id']) else row['country_id'],
    axis=1)

In [47]:
df.head(20)

,date,year,city,state_id,country_id,country_name,shape_type,duration_in_sec,duration_string,comments,date_posted,latitude,longitude
0,1949-10-10,1949,San Marcos,TX,US,United States,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10,1949,Lackland Afb,TX,US,United States,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10,1955,Chester,None,GB,United Kingdom,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10,1956,Edna,TX,US,United States,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10,1960,Kaneohe,HI,US,United States,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
5,1961-10-10,1961,Bristol,TN,US,United States,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889
6,1965-10-10,1965,Penarth,None,GB,United Kingdom,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2006-02-14,51.434722,-3.180000
7,1965-10-10,1965,Norwalk,CT,US,United States,circle,1200.0,20 minutes,A bright orange color changing to reddish colo...,1999-10-02,41.117500,-73.408333
8,1966-10-10,1966,Pell City,AL,US,United States,circle,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,2009-03-19,33.586111,-86.286111
9,1966-10-10,1966,Live Oak,FL,US,United States,circle,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,2005-05-11,30.294722,-82.984167


In [48]:
#kolejny check % obserwacji bez country_id
liczba_nan = df['country_id'].isna().sum()

liczba_wszystkich = len(df)

procent_nan = (liczba_nan / liczba_wszystkich) * 100

print(f'Liczba wierszy z NaN w kolumnie country_id: {liczba_nan}')
print(f'To stanowi {procent_nan:.2f}% wszystkich obserwacji.')

Liczba wierszy z NaN w kolumnie country_id: 617
To stanowi 0.73% wszystkich obserwacji.


In [49]:
#usuwanie pozycji gdzie country_id nie zostało uzupełnione
df= df.dropna(subset=['country_id'])


In [50]:
df2=pd.read_csv(r"C:\Repos\ufo_project\data\final_file\regions.csv",
                sep=",",
                header=0
                  )
df2.head(3)

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN


In [51]:
df_merged=df.merge(right=df2,how='left',left_on='country_id',right_on='alpha-2')

In [52]:
df_merged.head(20)

,date,year,city,state_id,country_id,country_name,shape_type,duration_in_sec,duration_string,comments,...,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,1949-10-10,1949,San Marcos,TX,US,United States,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
1,1949-10-10,1949,Lackland Afb,TX,US,United States,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
2,1955-10-10,1955,Chester,None,GB,United Kingdom,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,...,GB,GBR,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
3,1956-10-10,1956,Edna,TX,US,United States,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
4,1960-10-10,1960,Kaneohe,HI,US,United States,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
5,1961-10-10,1961,Bristol,TN,US,United States,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
6,1965-10-10,1965,Penarth,None,GB,United Kingdom,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,...,GB,GBR,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
7,1965-10-10,1965,Norwalk,CT,US,United States,circle,1200.0,20 minutes,A bright orange color changing to reddish colo...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
8,1966-10-10,1966,Pell City,AL,US,United States,circle,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
9,1966-10-10,1966,Live Oak,FL,US,United States,circle,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,...,US,USA,840.0,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN


In [53]:
df_merged.drop(columns="country-code",inplace=True)
df_merged.drop(columns="iso_3166-2",inplace=True)
df_merged.drop(columns="intermediate-region",inplace=True)
df_merged.drop(columns="region-code",inplace=True)
df_merged.drop(columns="sub-region-code",inplace=True)
df_merged.drop(columns="intermediate-region-code",inplace=True)

In [54]:
cleared_data = df_merged

In [55]:
cleared_data.to_csv(r"C:\Repos\ufo_project\data\cleared_data.csv",index=False)
cleared_data.to_parquet(r"C:\Repos\ufo_project\data\cleared_data.parquet",index=False)